In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta2

In [2]:
#!import Plugins/CustomPlugin/CompanySearchPlugin.cs

In [3]:
using Microsoft.SemanticKernel;;

In [4]:

// using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planners;

In [5]:
IKernel kernel = new KernelBuilder().WithAzureChatCompletionService("GPT3Model", "Your Azure OpenAI Service Endpoint", "Your Azure OpenAI Service API Key").Build();

In [6]:
var planner = new SequentialPlanner(kernel);

In [7]:
// var planner = kernel.ImportSkill(new SequentialPlanner(kernel));

In [8]:
planner

Microsoft.SemanticKernel.Planners.SequentialPlanner

In [9]:

CompanySearchPlugin customSkill = new CompanySearchPlugin();

In [10]:
var custom_skill = kernel.ImportFunctions(customSkill, "CustomSkill");

In [11]:
var skillsDirectory =  System.IO.Directory.GetCurrentDirectory() + "/Plugins";

In [12]:
var write_skill = kernel.ImportSemanticFunctionsFromDirectory(skillsDirectory,"WritePlugin");
var email_skill = kernel.ImportSemanticFunctionsFromDirectory(skillsDirectory,"EmailPlugin");

In [13]:
var plan = await planner.CreatePlanAsync("查找广州天气把结果翻译成中文后根据天气情况生成穿衣提示，并结合天气结果和穿衣提示写一封邮件");

In [14]:
// plan.Variables.ToPlan().PlanString
plan.Steps

index value 0 Microsoft.SemanticKernel.Planning.Plan State (empty) Steps (empty) Parameters key value text Guangzhou Outputs [ WEATHER_RESULT ] HasNextStep False NextStepIndex 0 Name WeatherSearch PluginName CustomSkill Description search weather RequestSettings <null> SkillName CustomSkill IsSemantic True 1 Microsoft.SemanticKernel.Planning.Plan State (empty) Steps (empty) Parameters key value input $WEATHER_RESULT Outputs [ TRANSLATED_WEATHER ] HasNextStep False NextStepIndex 0 Name Transalte PluginName WritePlugin Description 翻译 RequestSettings Microsoft.SemanticKernel.AI.AIRequestSettings ServiceId <null> ModelId <null> ExtensionData key value max_tokens 512 temperature 0.5 presence_penalty 0 frequency_penalty 0 SkillName WritePlugin IsSemantic True 2 Microsoft.SemanticKernel.Planning.Plan State (empty) Steps (empty) Parameters key value input $WEATHER_RESULT Outputs [ CLOTHING_TIPS ] HasNextStep False NextStepIndex 0 Name Tips PluginName WritePlugin Description 穿衣提示 RequestSettings Microsoft.SemanticKernel.AI.AIRequestSettings ServiceId <null> ModelId <null> ExtensionData key value max_tokens 512 temperature 0.5 presence_penalty 0 frequency_penalty 0 SkillName WritePlugin IsSemantic True 3 Microsoft.SemanticKernel.Planning.Plan State (empty) Steps (empty) Parameters key value input Weather: $TRANSLATED_WEATHER, Clothing Tips: $CLOTHING_TIPS Outputs [ RESULT__EMAIL ] HasNextStep False NextStepIndex 0 Name WeatherMail PluginName EmailPlugin Description Write an email about weather RequestSettings Microsoft.SemanticKernel.AI.AIRequestSettings ServiceId <null> ModelId <null> ExtensionData key value max_tokens 1024 temperature 0 top_p 0 presence_penalty 0 frequency_penalty 0 SkillName EmailPlugin IsSemantic True

In [15]:
plan.ToJson()

{"state":[],"steps":[{"state":[],"steps":[],"parameters":[{"Key":"text","Value":"Guangzhou"}],"outputs":["WEATHER_RESULT"],"next_step_index":0,"name":"WeatherSearch","plugin_name":"CustomSkill","description":"search weather"},{"state":[],"steps":[],"parameters":[{"Key":"input","Value":"$WEATHER_RESULT"}],"outputs":["TRANSLATED_WEATHER"],"next_step_index":0,"name":"Transalte","plugin_name":"WritePlugin","description":"\u7FFB\u8BD1"},{"state":[],"steps":[],"parameters":[{"Key":"input","Value":"$WEATHER_RESULT"}],"outputs":["CLOTHING_TIPS"],"next_step_index":0,"name":"Tips","plugin_name":"WritePlugin","description":"\u7A7F\u8863\u63D0\u793A"},{"state":[],"steps":[],"parameters":[{"Key":"input","Value":"Weather: $TRANSLATED_WEATHER, Clothing Tips: $CLOTHING_TIPS"}],"outputs":["RESULT__EMAIL"],"next_step_index":0,"name":"WeatherMail","plugin_name":"EmailPlugin","description":"Write an email about weather"}],"parameters":[],"outputs":["RESULT__EMAIL"],"next_step_index":0,"name":"planecaf1e34

In [16]:
var result = await kernel.RunAsync(plan);

In [17]:
result.GetValue<string>()

主题：关于广州天气和穿衣提示的出行提醒

亲爱的同事，

我希望这封邮件能够提醒您关于广州的天气情况和相应的穿衣建议。根据最新的天气预报，广州的温度为2度，且有雨。因此，我强烈建议您在出行时穿上暖和的衣物，以确保您的舒适度和健康。

以下是一些建议的穿衣方式：

1. 厚外套：请确保您穿上一件厚外套，以保持身体的温暖。这将有助于防止寒冷空气对您的影响。

2. 毛衣：在厚外套下面，您可以选择穿上一件毛衣。毛衣可以提供额外的保暖效果，使您感到更加舒适。

3. 长裤：为了保持双腿的温暖，我建议您穿上长裤。这将有助于防止寒冷空气对您的腿部造成不适。

此外，我还建议您携带雨具，以防止雨水淋湿身体。您可以考虑携带雨伞或雨衣，这将在下雨时为您提供保护。

最后，我想提醒您随时留意出行状况。由于天气的不确定性，可能会出现交通延误或其他不便之处。请确保您关注当地的交通状况和天气预报，并做好相应的准备。

如果您有任何其他问题或需要进一步的帮助，请随时与我联系。祝您旅途愉快！

最诚挚的问候，
[您的名字]